# Bias Buccaneers Image Recognition Challenge: Supervised Solution

In [ ]:
#import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image
from sklearn.utils import class_weight

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [ ]:
#create a directory for saving the trained models
os.mkdir("saved_models")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Prepare the data

In [ ]:
dataset_path = "../input/bias-buccaneers/"
train_df = pd.read_csv(f"{dataset_path}train/labels.csv")
test_df = pd.read_csv(f"{dataset_path}test/labels.csv")

In [ ]:
categories = train_df.columns[1:].tolist()
print(categories)

There are around 3730 samples in the training set which are unlabeled.

In [ ]:
train_df_labeled = train_df[train_df["skin_tone"].notna()].copy(deep=True) # take only labeled data

In [ ]:
#Modify the index values of the df
train_df_labeled.reset_index(drop=True, inplace=True)

Now we need to encode the labels into an integer such that we can use PyTorch's CrossEntropyLoss.

In [ ]:
skin_tone_labels = [f"monk_{i}" for i in range(1,11)]
gender_labels = ["male", "female"]
age_labels = ["0_17", "18_30", "31_60", "61_100"]

In [ ]:
#encode train samples
train_df_labeled['skin_tone'].replace(skin_tone_labels, list(range(len(skin_tone_labels))), inplace=True)
train_df_labeled['gender'].replace(gender_labels, list(range(len(gender_labels))), inplace=True)
train_df_labeled['age'].replace(age_labels, list(range(len(age_labels))), inplace=True)

In [ ]:
#encode test samples
test_df['skin_tone'].replace(skin_tone_labels, list(range(len(skin_tone_labels))), inplace=True)
test_df['gender'].replace(gender_labels, list(range(len(gender_labels))), inplace=True)
test_df['age'].replace(age_labels, list(range(len(age_labels))), inplace=True)

In [ ]:
#there are around 5 grayscale images in the dataset, so it is better to remove them.
grayscale_img_indices = [153, 647, 4905, 6184, 6389]
train_df_labeled.drop(labels=grayscale_img_indices, axis=0, inplace=True)
train_df_labeled.reset_index(drop=True, inplace=True)

### Building the DataLoader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, data_path, image_transform=None):
        self.df = df
        self.data_path = data_path
        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img = Image.open(f"{self.data_path}{self.df['name'][index]}")
        if self.image_transform:
            img = self.image_transform(img)
        #we need to provide labels for skin_tone, gender, age
        labels = (self.df['skin_tone'][index], self.df['gender'][index], self.df['age'][index])
        return img, labels

In [ ]:
#the dataset size is too small, it overfits to the training set, so we need to make use of data augmentations
image_transform = transforms.Compose([ transforms.ToTensor(),
                                       #transforms.Resize(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomRotation(20, interpolation=transforms.InterpolationMode.BILINEAR),
                                       transforms.RandomGrayscale(p=0.2),
                                       transforms.GaussianBlur(kernel_size=9, sigma=(0.1, 0.5)),
                                       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                       ])

train_dataset = ImageDataset(train_df_labeled, f"{dataset_path}train/", image_transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

## Build the model

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(nn.ReLU(), nn.Linear(1000, 512), nn.ReLU(), nn.Linear(512, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Helper functions

In [ ]:
def train(dataloader, model, loss_fn, optimizer, class_index:int, num_epochs:int):
    model.train()
    for epoch in range(1,num_epochs+1):
        running_loss = 0
        for imgs, labels in dataloader:
            labels = labels[class_index]
            output = model(imgs.to(device))
            loss = loss_fn(output, labels.to(device))
                   
            running_loss += loss.item()
        
            optimizer.zero_grad()
            loss.backward() 
            optimizer.step() 
             
        avg_loss = running_loss/len(dataloader) # Average loss for a single batch
        print(f'Epoch {epoch}/{num_epochs} - loss: {avg_loss:.2f}')

In [ ]:
def make_predictions(dataloader, model):
    model.eval()
    predicted_labels = []
    with torch.no_grad():
        for imgs, _ in dataloader:
            output = model(imgs.to(device))
            preds = output.argmax(dim=1).cpu().detach().tolist()
            predicted_labels.extend(preds)
    return predicted_labels

## Train the model

Train all the classifier models. Before doing so, the classes for each category is unbalanced, so we need to calcualte the class weights.

In [ ]:
#skintone classification
skintone_classifier = ClassifierModel(num_classes=10).to(device)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.array(range(len(skin_tone_labels))), y=train_df_labeled['skin_tone'].values)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
loss_fn   = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.SGD(skintone_classifier.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
train(train_loader, skintone_classifier, loss_fn, optimizer, class_index=0, num_epochs=30)
#save the trained model
skintone_classifier = skintone_classifier.cpu()
torch.save(skintone_classifier,"./saved_models/skintone_classifier.pt")

In [ ]:
#gender classification
gender_classifier = ClassifierModel(num_classes=2).to(device)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.array(range(len(gender_labels))), y=train_df_labeled['gender'].values)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
loss_fn   = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.SGD(gender_classifier.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
train(train_loader, gender_classifier, loss_fn, optimizer, class_index=1, num_epochs=30)
gender_classifier = gender_classifier.cpu()
torch.save(gender_classifier,"./saved_models/gender_classifier.pt")

In [ ]:
#age classification
age_classifier = ClassifierModel(num_classes=4).to(device)
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.array(range(len(age_labels))), y=train_df_labeled['age'].values)
class_weights = torch.tensor(class_weights,dtype=torch.float).to(device)
loss_fn   = nn.CrossEntropyLoss(weight=class_weights) #CrossEntropyLoss with class_weights
optimizer = torch.optim.SGD(age_classifier.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
train(train_loader, age_classifier, loss_fn, optimizer, class_index=2, num_epochs=30)
age_classifier = age_classifier.cpu()
torch.save(age_classifier,"./saved_models/age_classifier.pt")

## Make Predictions on Test data

In [ ]:
test_image_transform = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                         ])

In [ ]:
#prepare the dataloader for test data
test_data = ImageDataset(test_df, f"{dataset_path}test/", test_image_transform)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

In [ ]:
#skintone_predictions
skintone_classifier = torch.load("./saved_models/skintone_classifier.pt").to(device)
skintone_predictions = make_predictions(test_loader, skintone_classifier)
del skintone_classifier

In [ ]:
#gender_predictions
gender_classifier = torch.load("./saved_models/gender_classifier.pt").to(device)
gender_predictions = make_predictions(test_loader, gender_classifier)
del gender_classifier

In [ ]:
#age_predictions
age_classifier = torch.load("./saved_models/age_classifier.pt").to(device)
age_predictions = make_predictions(test_loader, age_classifier)
del age_classifier

In [ ]:
predictions = [skintone_predictions, gender_predictions, age_predictions]

## Prepare Submission

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# calculate accuracy
acc = {}
for i in range(3):
    icat = categories[i]
    iacc = accuracy_score(test_df[icat], predictions[i])
    acc[icat] = iacc

# calculate disparity
def disparity_score(ytrue, ypred):
    cm = confusion_matrix(ytrue,ypred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    all_acc = list(cm.diagonal())
    return max(all_acc) - min(all_acc)

disp = {}
for i in range(3):
    icat = categories[i]
    idisp = disparity_score(test_df[icat], predictions[i])
    disp[icat] = idisp
disp

results = {'accuracy': acc, 'disparity': disp}
results

In [ ]:
def getScore(results):
    acc = results['accuracy']
    disp = results['disparity']
    ad = 2*acc['gender']*(1-disp['gender']) + 4*acc['age']*(1-disp['age']**2) + 10*acc['skin_tone']*(1-disp['skin_tone']**5)
    return ad

title = 'Test Submission'
    
submission = {
    'submission_name': title,
    'score': getScore(results),
    'metrics': results
}
submission

In [ ]:
import json
with open("balaji_submission.json", "w") as f:
    json.dump(submission, f, indent=4)

---